In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import h5py
import copy

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, SequentialSampler
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

from torchvision import transforms

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

np.random.seed(0)
torch.manual_seed(0)

In [ ]:
features_filepath = '/content/drive/MyDrive/LakeRegression/340_Veri_toplam_temiz.xlsx'
feats_df = pd.read_excel(features_filepath)
selected_feature = 'Klorofil-a (µg/L)'
feats_df = feats_df[['Date', 'Station', selected_feature, 'X', 'Y']]
feats_df.head()

In [ ]:
scaler = StandardScaler()
feats_df[selected_feature] = scaler.fit_transform(feats_df[selected_feature].values.reshape(-1, 1))
feats_df.head()

In [ ]:
import os
import h5py
import numpy as np
from tqdm import tqdm

np.random.seed(0)

unique_dates = feats_df['Date'].unique()

X_train = []
y_train = []
X_val = []
y_val = []
X_test = []
y_test = []
patch_size = 9
for i in tqdm(range(1, 35)):
    if i != 22 and i != 23:
        folder_path = f'/content/drive/MyDrive/LakeRegression/data/{i}'
        for j in range(10):
            file_path = f'{folder_path}/station_{j}_{patch_size}.h5'
            if os.path.isfile(file_path):
                with h5py.File(file_path, 'r') as f:
                    a_group_key = list(f.keys())[0]
                    image = np.array(f[a_group_key])

                    unique_date = unique_dates[i - 1]
                    unique_station = j + 1
                    label = feats_df.loc[(feats_df['Date'] == unique_date) & (feats_df['Station'] == unique_station)][selected_feature].values[0]

                    date = pd.to_datetime(unique_date)
                    if date < pd.to_datetime('2019-03-15'):
                        X_train.append(image)
                        y_train.append(label)
                    elif date >= pd.to_datetime('2019-03-15') and date < pd.to_datetime('2019-05-01'):
                        X_val.append(image)
                        y_val.append(label)
                    else:
                        X_test.append(image)
                        y_test.append(label)

print(f'Train % {len(X_train) / (len(X_train) + len(X_val) + len(X_test))} | Val % {len(X_val) / (len(X_train) + len(X_val) + len(X_test))} | Test % {len(X_test) / (len(X_train) + len(X_val) + len(X_test))}')

X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

print(f'Train shape: {X_train.shape} | {y_train.shape} | Val shape: {X_val.shape} | {y_val.shape} | Test shape: {X_test.shape} | {y_test.shape}')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, SequentialSampler
from torchvision import transforms

torch.manual_seed(0)

class LakeDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)
        self.transform = transform

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

means = []
stds = []
for i in range(12):
    means.append(np.mean(X_train[:, i, :, :]))
    stds.append(np.std(X_train[:, i, :, :]))

transform = transforms.Compose([
    transforms.Lambda(lambda x: torch.from_numpy(x).float()),
    transforms.Normalize(means, stds)
])

train_dataset = LakeDataset(X_train, y_train, transform=transform)
val_dataset = LakeDataset(X_val, y_val, transform=transform)
test_dataset = LakeDataset(X_test, y_test, transform=transform)

# use sequential sampler to preserve the date order
train_loader = DataLoader(train_dataset, batch_size=32, sampler=SequentialSampler(train_dataset))
val_loader = DataLoader(val_dataset, batch_size=32, sampler=SequentialSampler(val_dataset))
test_loader = DataLoader(test_dataset, batch_size=32, sampler=SequentialSampler(test_dataset))

print(f'Train loader: {len(train_loader)} | Val loader: {len(val_loader)} | Test loader: {len(test_loader)}')

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [ ]:
def squash(inputs):
    squared_norm = (inputs ** 2).sum(-1, keepdim=True)
    output = squared_norm * inputs / ((1. + squared_norm) * torch.sqrt(squared_norm))
    return output

class PrimaryCapsules(nn.Module):
    def __init__(self, num_capsules=8, in_channels=12, out_channels=32, kernel_size=9, num_routes=32 * 6 * 6):
        super(PrimaryCapsules, self).__init__()

        self.capsules = nn.ModuleList([
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=2, padding=0)
            for _ in range(num_capsules)])

    def forward(self, x):
        u = [capsule(x) for capsule in self.capsules]
        u = torch.stack(u, dim=1)
        u = u.view(x.size(0), 32 * 6 * 6, -1)
        return squash(u)


class DigitCaps(nn.Module):
    def __init__(self, num_capsules=1, num_routes=32 * 6 * 6, in_channels=8, out_channels=16):
        super(DigitCaps, self).__init__()

        self.in_channels = in_channels
        self.num_routes = num_routes
        self.num_capsules = num_capsules

        self.W = nn.Parameter(torch.randn(1, num_routes, num_capsules, out_channels, in_channels))

    def forward(self, x):
        batch_size = x.size(0)
        x = torch.stack([x] * self.num_capsules, dim=2).unsqueeze(4)

        W = torch.cat([self.W] * batch_size, dim=0)
        u_hat = torch.matmul(W, x)

        b_ij = Variable(torch.zeros(1, self.num_routes, self.num_capsules, 1))
        if torch.cuda.is_available():
            b_ij = b_ij.cuda()

        num_iterations = 3
        for iteration in range(num_iterations):
            c_ij = F.softmax(b_ij, dim=1)
            s_j = (c_ij * u_hat).sum(dim=1, keepdim=True)
            v_j = squash(s_j)

            if iteration < num_iterations - 1:
                a_ij = torch.matmul(u_hat.transpose(3, 4), torch.cat([v_j] * self.num_routes, dim=1))
                b_ij = b_ij + a_ij.squeeze(4).mean(dim=0, keepdim=True)

        return v_j.squeeze(1)


class CapsNet(nn.Module):
    def __init__(self):
        super(CapsNet, self).__init__()
        self.conv_layer = nn.Conv2d(in_channels=12, out_channels=256, kernel_size=9, stride=1)
        self.primary_capsules = PrimaryCapsules()
        self.digit_capsules = DigitCaps()
        self.decoder = nn.Linear(16 * 1, 1) 

    def forward(self, data):
        output = self.conv_layer(data)
        output = self.primary_capsules(output)
        output = self.digit_capsules(output)
        predictions = self.decoder(output)
        return predictions

In [ ]:
def train(model, train_loader, val_loader, criterion, optimizer, epochs, device): 
    print(f'Using device: {device}')
    for epoch in range(epochs):
        train_loss = 0.0
        train_predictions = []
        train_actuals = []
        model.train()
        for X, y in train_loader:
            X = X.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_predictions.extend(y_hat.cpu().detach().numpy())
            train_actuals.extend(y.cpu().numpy())
        train_loss /= len(train_loader)

        train_r2 = r2_score(train_actuals, train_predictions)
        train_mse = mean_squared_error(train_actuals, train_predictions)
        train_rmse = mean_squared_error(train_actuals, train_predictions, squared=False)

        val_loss = 0.0
        val_predictions = []
        val_actuals = []

        model.eval()
        with torch.no_grad():
            for X, y in val_loader:
                X = X.to(device)
                y = y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y.unsqueeze(1))
                val_loss += loss.item()
                val_predictions.extend(y_hat.cpu().detach().numpy())
                val_actuals.extend(y.cpu().numpy())
        val_loss /= len(val_loader)

        val_r2 = r2_score(val_actuals, val_predictions)
        val_mse = mean_squared_error(val_actuals, val_predictions)
        val_rmse = mean_squared_error(val_actuals, val_predictions, squared=False)

        print(f'Epoch {epoch + 1}/{epochs} | Train loss: {train_loss:.4f} R2: {train_r2:.4f} MSE: {train_mse:.4f} RMSE: {train_rmse:.4f} | Val loss: {val_loss:.4f} R2: {val_r2:.4f} MSE: {val_mse:.4f} RMSE: {val_rmse:.4f}')
   
    return model

capsule_net = CapsNet().to(device)

criterion = nn.MSELoss()  
optimizer = optim.Adam(capsule_net.parameters())

epochs = 50